<a href="https://colab.research.google.com/github/ernestopalacios/eerssa/blob/main/GestionOT/recopilar_ot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gather and Merge "Ordenes de Trabajo"
Date: 01/07/2024
Author: Ernesto Palacios
epalacios@eerssa.gob.ec

Development book:

https://ipywidgets.readthedocs.io/en/stable/

In [1]:
# Librerias requeridas:

%pip install --upgrade pymupdf
%pip install PyPDF2
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Import Libraries
import os
import logging
from pprint import pprint
from PyPDF2 import PdfWriter, PdfReader, PdfFileMerger

import pandas as pd

logging.basicConfig(level=logging.DEBUG)

# Define a function that takes the name of the folder and outputs
# what the resulting PDF file should be named.

def getPDFName( current_folder ):
    try:
        data = current_folder.split()
        position = data[0]
        cuadrilla = data [1] # not used
        nombre = " ".join(data[2:]).title()
    except:
        print("No se ha podido extraer el nombre de Cuadrilla")
        position = '00'
        nombre = 'Sin Nombre'

    respuesta = " ".join( [position, nombre] )

    return respuesta

# Array with the names of the PDF files inside each subDirectory (Cuadrilla)

def getPDFListNames( root, path ):
    try:
        abs_path = os.path.join( root, path )
        pdf_files = sorted( [f for f in os.listdir(abs_path) if f.endswith('.pdf')] ) # Gemini
        path_files = sorted( [ os.path.join( abs_path, item ) for item in pdf_files ] )
    except:
        print("No es un directorio válido")
        return []

    return path_files


# create & open the resuling file
# Open the current file
# Check the number of pages of the file
# Strip the first page, and save the ohther pages to the file

def unirPDFs( listado, nombrePDFUnido):
    try:
        merger = PdfWriter()
        for pdf in listado:
            pdf_reader = PdfReader(pdf)
            num_pages = len(pdf_reader.pages)

            if num_pages > 2:
                merger.append(pdf, [1,2])
            else:
                merger.append(pdf,[1])

        merger.write( nombrePDFUnido )
        merger.close()
        success = True
    except:
        success = False
    return success


In [7]:
# https://medium.com/@mgkyawzayya/splitting-and-merging-pdf-files-with-python-using-pypdf2-cfce5c948c36


root_dir = "/content/drive/MyDrive/Colab_Disk/OT/2024/09 SEPTIEMBRE"
#root_dir = "D:\\INFORMES\\EERSSA\\ACTIVIDADES DIARIAS DE TRABAJO DE LAS CUADRILLAS\\2024\\09 SEPTIEMBRE"

try:
    nn_mesActual, mes_actual = os.path.basename(root_dir).split()
    prefijo_nombrePDF = nn_mesActual + " " + mes_actual.lower() + " - "
    prefijo_nombrePDF = os.path.join( root_dir, prefijo_nombrePDF ) # Aboslute/Relative Path
except: # Place holder in case diffent directory name
    nn_mesActual = '00'
    mes_actual = 'NombreMes'


try:
    folders = sorted ( [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))] )
except:
    print("El directorio no es válido.")

pprint(folders)
# Al final de esta operación espero obtener una lista con las carpetas de las Cuadrillas.

['01 JEFE ZONAL ZAMORA',
 '02 CUADRILLA ZAMORA',
 '03 CUADRILLA ALUMBRADO ZAMORA',
 '04 CUADRILLA YACUAMBI',
 '05 CUADRILLA YANTZAZA',
 '06 CUADRILLA ENERGIZADA YANTZAZA',
 '07 CUADRILLA GUAYZIMI',
 '08 CUADRILLA EL PANGUI',
 '09 CUADRILLA GUALAQUIZA',
 '21 (electricistas) Ag. Zamora',
 '22 (electricistas) Ag. Yantzaza',
 '23 (electricistas) Ag. El Pangui',
 '24 (electricistas) Ag. Gualaquiza']


In [8]:
# Crear un Pandas.Dataframe organizando la información:

df = pd.DataFrame( folders, columns = ["carpetas"])
df["nombre_pdf_unificado"] = prefijo_nombrePDF + df["carpetas"].apply( lambda x: getPDFName(x) ) + ".pdf"
df["listaPDFs"] = df["carpetas"].apply( lambda x: getPDFListNames( root_dir, x ) )
df["estadoMergePDF"] = df.apply( lambda row: unirPDFs( row['listaPDFs'],row['nombre_pdf_unificado']), axis = 1)
df

,carpetas,nombre_pdf_unificado,listaPDFs,estadoMergePDF
0,01 JEFE ZONAL ZAMORA,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
1,02 CUADRILLA ZAMORA,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
2,03 CUADRILLA ALUMBRADO ZAMORA,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
3,04 CUADRILLA YACUAMBI,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
4,05 CUADRILLA YANTZAZA,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
5,06 CUADRILLA ENERGIZADA YANTZAZA,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
6,07 CUADRILLA GUAYZIMI,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
7,08 CUADRILLA EL PANGUI,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
8,09 CUADRILLA GUALAQUIZA,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True
9,21 (electricistas) Ag. Zamora,/content/drive/MyDrive/Colab_Disk/OT/2024/09 S...,[/content/drive/MyDrive/Colab_Disk/OT/2024/09 ...,True



-----

Develop

## Exploring the os library

In [ ]:
import os

anio_dir = "C:\\Users\\epalacios\\OneDrive\\01 JEZO\\01 ACTIVIDADES DIARIAS DE TRABAJO DE LAS AGENCIAS\\2024\\01 ENERO"

def get_directory_depth(start_path=anio_dir):
    depth = 0
    for root, dirs, files in os.walk(start_path):
        depth = max(depth, root.count(os.sep) - start_path.count(os.sep))
    return depth

depth = get_directory_depth()
print("Depth of current directory:", depth)

Depth of current directory: 1
